<a href="https://colab.research.google.com/github/Abdullah-Mehboob1/Abdullah-Mehboob1/blob/main/citrusfirstsucces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Load the dataset
dataset_path = '/content/drive/MyDrive/citrusdatasets/Citrus'
image_size = (224, 224)
batch_size = 32

# Load the dataset into a pandas dataframe
import pandas as pd
df = pd.DataFrame({'filename': os.listdir(dataset_path),
                   'label': [0 if 'Healthy' in filename else 1 if 'Black Spot' in filename else 2 if 'Canker' in filename else 3 if 'Greening' in filename else 4 for filename in os.listdir(dataset_path)]})


In [27]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a list to store the file paths and labels
file_paths = []
labels = []

# Iterate over the directories and files
for dir_name in os.listdir('/content/drive/MyDrive/citrusdatasets/Citrus'):
    dir_path = os.path.join('/content/drive/MyDrive/citrusdatasets/Citrus', dir_name)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        file_paths.append(file_path)
        labels.append(dir_name)

# Create a dataframe with the file paths and labels
df = pd.DataFrame({'file_path': file_paths, 'label': labels})

# Split the dataframe into training, validation, and testing sets
train_df, val_test_df = train_test_split(df, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

# Create an ImageDataGenerator for the training set
train_datagen = ImageDataGenerator(rescale=1./255)

# Create a flow from the training dataframe
train_flow = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Create an ImageDataGenerator for the validation set
val_datagen = ImageDataGenerator(rescale=1./255)

# Create a flow from the validation dataframe
val_flow = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Create an ImageDataGenerator for the testing set
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a flow from the testing dataframe
test_flow = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 356 validated image filenames belonging to 4 classes.
Found 119 validated image filenames belonging to 4 classes.
Found 119 validated image filenames belonging to 4 classes.


In [13]:
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [28]:
from sklearn.preprocessing import LabelEncoder

# Fit the LabelEncoder on the combined labels from all sets
le = LabelEncoder()
all_labels = pd.concat([train_df['label'], val_df['label'], test_df['label']])
le.fit(all_labels)

# Now transform the labels in each set
train_df['label'] = le.transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])
test_df['label'] = le.transform(test_df['label'])

In [30]:
# Create data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Convert the label column to string type before creating generators
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

train_generator = train_datagen.flow_from_dataframe(train_df, x_col='file_path', y_col='label', target_size=image_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_dataframe(val_df, x_col='file_path', y_col='label', target_size=image_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_dataframe(test_df, x_col='file_path', y_col='label', target_size=image_size, batch_size=batch_size, class_mode='categorical')

Found 356 validated image filenames belonging to 4 classes.
Found 119 validated image filenames belonging to 4 classes.
Found 119 validated image filenames belonging to 4 classes.


In [32]:
# Define the CNN model
model = keras.Sequential([
    keras.layers.Input(shape=(image_size[0], image_size[1], 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])

In [33]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [34]:
# Train the model
history = model.fit(train_generator, epochs=10, validation_data=val_generator)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 192s 11s/step - accuracy: 0.2871 - loss: 2.2421 - val_accuracy: 0.3529 - val_loss: 1.2714
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.4108 - loss: 1.2406 - val_accuracy: 0.3866 - val_loss: 1.1977
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - accuracy: 0.5027 - loss: 1.0844 - val_accuracy: 0.3950 - val_loss: 1.3687
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.4023 - loss: 1.1942 - val_accuracy: 0.5798 - val_loss: 0.9442
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.6148 - loss: 0.8840 - val_accuracy: 0.6218 - val_loss: 0.8369
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.5821 - loss: 0.8536 - val_accuracy: 0.6134 - val_loss: 0.7868
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.5101 - loss: 0.8732 - val_accuracy: 0.6555 - val_loss: 0.8549
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.5899 - loss: 0.8932 - val_accuracy: 0.4874 - val_loss: 0.9585
E

In [35]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 7s/step - accuracy: 0.7643 - loss: 0.7287
Test accuracy: 0.73


In [36]:
# Fine-tune the model
model.fit(train_generator, epochs=5, validation_data=val_generator)


Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.6023 - loss: 0.8198 - val_accuracy: 0.5126 - val_loss: 0.8480
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.5919 - loss: 0.7761 - val_accuracy: 0.6807 - val_loss: 0.8023
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.7061 - loss: 0.7136 - val_accuracy: 0.6218 - val_loss: 0.7772
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.6910 - loss: 0.6894 - val_accuracy: 0.6134 - val_loss: 0.8550
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.5871 - loss: 0.8254 - val_accuracy: 0.6471 - val_loss: 0.7403


In [37]:
# Evaluate the model again
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 916ms/step - accuracy: 0.6740 - loss: 0.7262
Test accuracy: 0.71


In [38]:
# Two-stage model
# Region Proposal Network (RPN)
rpn_model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])


In [39]:
# Classifier
classifier_model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])


In [40]:
# Compile the model
classifier_model.compile(optimizer='adam',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])


In [41]:
# Train the model
history = classifier_model.fit(train_generator,
                               epochs=10,
                               validation_data=val_generator)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.3010 - loss: 2.4955 - val_accuracy: 0.4790 - val_loss: 1.2335
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.4137 - loss: 1.2291 - val_accuracy: 0.4874 - val_loss: 1.0504
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.4719 - loss: 1.0903 - val_accuracy: 0.5042 - val_loss: 1.2464
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 86s 4s/step - accuracy: 0.5463 - loss: 1.0163 - val_accuracy: 0.5378 - val_loss: 0.9718
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.5095 - loss: 0.9931 - val_accuracy: 0.5294 - val_loss: 1.0138
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 60s 5s/step - accuracy: 0.5613 - loss: 0.9424 - val_accuracy: 0.5546 - val_loss: 0.9899
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.5324 - loss: 1.0019 - val_accuracy: 0.5714 - val_loss: 0.9402
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.5582 - loss: 0.8993 - val_accuracy: 0.5798 - val_loss:

In [42]:
# Evaluate the model on the test set
test_loss, test_acc = classifier_model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')


4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6101 - loss: 0.7494
Test accuracy: 0.58


In [46]:
y_pred = classifier_model.predict(test_generator)
y_pred_class = np.argmax(y_pred, axis=1)
y_true = test_df['label']
unique_labels = test_df['label'].unique()

y_pred_class_labels = [unique_labels[i] for i in y_pred_class]

print(f'Test precision: {precision_score(y_true, y_pred_class_labels, average="macro", zero_division=0):.2f}')
print(f'Test recall: {recall_score(y_true, y_pred_class_labels, average="macro", zero_division=0):.2f}')
print(f'Test F1-score: {f1_score(y_true, y_pred_class_labels, average="macro", zero_division=0):.2f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Test precision: 0.21
Test recall: 0.34
Test F1-score: 0.23


In [45]:
y_pred = classifier_model.predict(test_generator)
y_pred_class = np.argmax(y_pred, axis=1)
y_true = test_df['label']
unique_labels = test_df['label'].unique()

y_pred_class_labels = [unique_labels[i] for i in y_pred_class]

print(f'Test precision: {precision_score(y_true, y_pred_class_labels, average="macro"):.2f}')
print(f'Test recall: {recall_score(y_true, y_pred_class_labels, average="macro"):.2f}')
print(f'Test F1-score: {f1_score(y_true, y_pred_class_labels, average="macro"):.2f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
Test precision: 0.18
Test recall: 0.22
Test F1-score: 0.17


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
